<a href="https://colab.research.google.com/github/eunjoo-ny/ML-Code/blob/main/9_Pytorch_lab_09_3_mnist_nn_xavier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
   torch.cuda.manual_seed_all(777)

In [ ]:
#parameters
learning_rate= 0.001
training_epochs =15
batch_size = 100


In [ ]:
#minst dataset
mnist_train =dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
#dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
#nn layers
linear1 = torch.nn.Linear(784,256,bias=True)
linear2 = torch.nn.Linear(256,256,bias=True)
linear3 = torch.nn.Linear(256,10,bias=True)
relu = torch.nn.ReLU()

In [ ]:
#xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0215, -0.0894,  0.0598,  ...,  0.0200,  0.0203,  0.1212],
        [ 0.0078,  0.1378,  0.0920,  ...,  0.0975,  0.1458, -0.0302],
        [ 0.1270, -0.1296,  0.1049,  ...,  0.0124,  0.1173, -0.0901],
        ...,
        [ 0.0661, -0.1025,  0.1437,  ...,  0.0784,  0.0977, -0.0396],
        [ 0.0430, -0.1274, -0.0134,  ..., -0.0582,  0.1201,  0.1479],
        [-0.1433,  0.0200, -0.0568,  ...,  0.0787,  0.0428, -0.0036]],
       requires_grad=True)

In [ ]:
#model
model=torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [ ]:
#define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
#Softmax is internally computed.
optimizer= torch.optim.Adam(model.parameters(),
                            lr=learning_rate)

In [ ]:
total_batch= len(data_loader)
for epoch in range(training_epochs):
    avg_cost=0

    for X,Y in data_loader:
      #reshape input image into[batch_size by 784]
      #label is not one_hot encoded
      X=X.view(-1,28*28).to(device)
      Y=Y.to(device)

      hypothesis=model(X)
      cost=criterion(hypothesis,Y)

      optimizer.zero_grad()
      cost.backward()
      optimizer.step()

      avg_cost +=cost/total_batch

    print('Epoch:{:4d}/{} ,cost:{:.9f}'.format(epoch+1,training_epochs,avg_cost))
print('Learning finished')

Epoch:   1/15 ,cost:0.007955992
Epoch:   2/15 ,cost:0.007448274
Epoch:   3/15 ,cost:0.012852008
Epoch:   4/15 ,cost:0.007400780
Epoch:   5/15 ,cost:0.008552849
Epoch:   6/15 ,cost:0.007573675
Epoch:   7/15 ,cost:0.007183088
Epoch:   8/15 ,cost:0.010006516
Epoch:   9/15 ,cost:0.007394472
Epoch:  10/15 ,cost:0.007263596
Epoch:  11/15 ,cost:0.005760915
Epoch:  12/15 ,cost:0.005910828
Epoch:  13/15 ,cost:0.004704008
Epoch:  14/15 ,cost:0.007410221
Epoch:  15/15 ,cost:0.005861710
Learning finished


In [ ]:
#Test the model using test sets
with torch.no_grad():
     X_test=mnist_test.test_data.view(-1,28*28).float().to(device)
     Y_test=mnist_test.test_labels.to(device)

     prediction=model(X_test)
     correction_prediction=torch.argmax(prediction,1)==Y_test
     accuracy = correction_prediction.float().mean()

     print('Accuracy:',accuracy.item())

#Get one and predict
     r= random.randint(0,len(mnist_test)-1)
     X_single_data=mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
     Y_single_data=mnist_test.test_labels[r:r+1].to(device)

     print('Label:',Y_single_data.item())
     single_prediction= model(X_single_data)
     print('Predcition:',torch.argmax(single_prediction,1).item())

Accuracy: 0.9800999760627747
Label: 8
Predcition: 8


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
